In [1]:
using Pkg
Pkg.activate(".")

  Activating new environment at `~/Documents/Github2/Physics-215-Julia/Session 2 - Measuring code performance/Project.toml`


### Elliptic Integral of the First Kind

### $K(k) = \int_{0}^{\frac{\pi}{2}} \frac{d\theta}{\sqrt{1 - k^2 \sin^2 \theta}}$

Before doing any integration we must define the function that takes care of our integration $\int f(x)dx$. The function below solves the N-point Gaussian approximation of our integral.

In [2]:
function gaussian_approximation(N)
    global a = collect(LinRange(3,4*N-1,N))/(4*N+2)
    global x = cos.(pi*a.+1 ./ (8*N*N*tan.(a)))
    
    epsilon = 1e-15
    global delta = 1.0
    while delta > epsilon
        p0 = ones(N)
        p1 = copy(x)
        for k in range(1,N-1, step = 1)
            p0, p1 = p1, ((2*k+1).* x .* p1 .-k .*p0)./ (k+1)
        end
        global dp = (N+1) * (p0 .-x .* p1) ./ (1 .-x.*x)
        dx = p1 ./ dp
        x = x - dx
        global delta = maximum(abs.(dx))
    end
    w = 2*(N+1)*(N+1)./(N*N.*(1 .-x.*x).*dp.*dp)
    return x, w
end 

gaussian_approximation (generic function with 1 method)

In [3]:
function gaussian_approximation(N,A,B)
    x,w = gaussian_approximation(N)
    return 0.5*(B-A) .* x .+ 0.5 .* (B+A), 0.5.* (B-A) .* w
end

gaussian_approximation (generic function with 2 methods)

In [4]:
function my_func(m,theta)
    return 1 / sqrt(1 - (m * sin(theta)^2))
end

my_func (generic function with 1 method)

In [5]:
function elliptic_integral_first_kind(N,m)
    A = 0
    B = pi/2

    x,w = gaussian_approximation(N,A,B)

    global integral = 0.
    for i in range(1,N, step = 1)
        integral += my_func(m, x[i]) * w[i]
    end
    return integral
end

elliptic_integral_first_kind (generic function with 1 method)

In [6]:
N = 100
m = 0
elliptic_integral_first_kind(N,m)

1.5707963267948966

In [7]:
using SpecialFunctions
using ProgressMeter

In [8]:
ellipk(m)

1.5707963267948966

In [22]:
function compute_accuracy()
    m_vals = -1:0.04:0.96
    N_vals = 100:100:5000
    
    n = 50
    accuracy_values = zeros(n,n)
    
    progress = Progress(n*n)
    for ii in 1:n*n
        xi, yi = divrem(ii-1, n) .+ (1,1)
        own_function = elliptic_integral_first_kind(convert(Int64, N_vals[yi]), m_vals[xi])
        builtin_function = ellipk(m_vals[xi])

        accuracy_values[xi,yi] = own_function - builtin_function
         next!(progress)
    end
    return accuracy_values
end

compute_accuracy (generic function with 1 method)

In [ ]:
accuracy_values = compute_accuracy()

Progress:   2%|▊                                        |  ETA: 0:12:52

In [11]:
using Plots

In [12]:
heatmap(abs.(accuracy_values), 
    xticks = (range(1,100, length = 5), range(-0.999,0.999, length = 5)), 
    yticks = (range(1,100, length = 10), range(10,1000, length = 10)), 
    c = :gist_yarg)

LoadError: UndefVarError: accuracy_values not defined

In [21]:
collect(-1:0.04:0.96)

50-element Vector{Float64}:
 -1.0
 -0.96
 -0.92
 -0.88
 -0.84
 -0.8
 -0.76
 -0.72
 -0.68
 -0.64
 -0.6
 -0.56
 -0.52
  ⋮
  0.52
  0.56
  0.6
  0.64
  0.68
  0.72
  0.76
  0.8
  0.84
  0.88
  0.92
  0.96

In [14]:
pushfirst!(collect(100:100:5000), 1)

51-element Vector{Int64}:
    1
  100
  200
  300
  400
  500
  600
  700
  800
  900
 1000
 1100
 1200
    ⋮
 3900
 4000
 4100
 4200
 4300
 4400
 4500
 4600
 4700
 4800
 4900
 5000

In [15]:
d = Vector{Float64}()
push!(d, 1.0)
push!(d, 2.0)

2-element Vector{Float64}:
 1.0
 2.0